<a href="https://colab.research.google.com/github/playpark95/Others-at-KoreaUniv/blob/main/KU_%EB%94%A5%EB%9F%AC%EB%8B%9D%EC%8A%A4%ED%84%B0%EB%94%94_4%EC%A3%BC%EC%B0%A8_ipynb%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/sungbinlim/KU-pytorch-study-2023.git

Cloning into 'KU-pytorch-study-2023'...
remote: Enumerating objects: 154, done.
remote: Counting objects: 100% (101/101), done.
remote: Compressing objects: 100% (41/41), done.
remote: Total 154 (delta 67), reused 92 (delta 60), pack-reused 53
Receiving objects: 100% (154/154), 60.06 MiB | 11.49 MiB/s, done.
Resolving deltas: 100% (89/89), done.
Updating files: 100% (14/14), done.


In [ ]:
!unzip ../content/KU-pytorch-study-2023/ion_data/train.zip -d ../content/KU-pytorch-study-2023/ion_data/
!unzip ../content/KU-pytorch-study-2023/ion_data/valid.zip -d ../content/KU-pytorch-study-2023/ion_data/
!rm -r -f ../content/KU-pytorch-study-2023/ion_data/__MACOSX/

In [ ]:
import os
import numpy as np
import torch
from torch.utils.data import Dataset
from torchvision.transforms import transforms as T
import pandas as pd
from PIL import Image # Bytes 로 압축된 이미지를 이미지로 변환하기 위해 필요

class IonDataset(Dataset):
    def __init__(self, data_dir, mode='train', img_size=(10, 300), n=None):
        self.df = pd.read_pickle(os.path.join(data_dir, f"{mode}.pkl"))

        if n is not None:
            self.df = self.df[self.df['n'] == n]
        self.data_dir = data_dir
        self.image_path = self.df['img_path'].values

        # min-max normalization
        self.min_a2 = 0.146
        self.max_a2 = 0.522
        self.df["norm_potential"] = self.df["potential"].apply(lambda p : (p - self.min_a2) / (self.max_a2 - self.min_a2))

        self.n = self.df["n"].values
        self.max_n = self.n.max()
        self.min_n = self.n.min()

        self.potential = self.df['potential'].values.astype('float32')
        self.norm_potential = self.df['norm_potential'].values.astype('float32')
        self.position = self.df['position'].values

        self.h, self.w = img_size

        self.img_transform = T.Compose([T.ToTensor(),
        ])

    def make_one_hot(self, scalar):
        one_hot_dim = self.max_n - self.min_n + 1
        address = scalar - one_hot_dim - 1
        one_hot_vec = np.zeros(one_hot_dim)
        one_hot_vec[address] = 1.0
        return one_hot_vec

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):

        img = Image.open(os.path.join(self.data_dir, self.image_path[idx]))
        img = self.img_transform(img)
        ion_number = torch.tensor(self.make_one_hot(self.n[idx]))
        ion_potential = torch.tensor(self.norm_potential[idx])

        return img, ion_number, ion_potential

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models

In [ ]:
class MLP(nn.Module):
    def __init__(self, hidden_variables=[128, 64], input_output_dim=(1, 1)):
        super().__init__()
        self.input_variable_dim = input_output_dim[0]
        self.output_variable_dim = input_output_dim[1]
        self.list_hidden_variable = hidden_variables
        self.layer = nn.Sequential()

        variable_dim = self.input_variable_dim
        for i, hidden_variable in enumerate(self.list_hidden_variable):
            self.layer.add_module('layer_' + str(i), nn.Linear(variable_dim, hidden_variable, dtype=torch.float32))
            self.layer.add_module('activation_' + str(i), nn.ReLU())
            variable_dim = hidden_variable
        self.layer.add_module('final_layer', nn.Linear(variable_dim, self.output_variable_dim, dtype=torch.float32))

    def forward(self, x):
        y_hat = self.layer(x)
        return y_hat

class IonPredictor(nn.Module):
    def __init__(self, kernel_size=3, num_features=1000):
        super().__init__()

        self.cnn = models.resnet18()
        self.cnn.conv1 = nn.Conv2d(1, 64, kernel_size=kernel_size, stride=2, padding=3, bias=False)
        self.mlp = MLP(input_output_dim=(num_features, 5))

    def forward(self, x):

        z = self.cnn(x)
        z.view(z.size(0), -1)
        predict = self.mlp(z)
        return predict[:, :4], predict[:, 4]